In [12]:
import polars as pl


In [13]:
(pitching :=
 pl.read_csv("./data/league_era.csv")
)


yearID,lgID,league_wide_ERA,DH Rule
i64,str,f64,str
2016,"""AL""",4.521031,"""DH"""
1993,"""NL""",4.521149,"""DH"""
1949,"""AL""",4.751302,"""NO DH"""
1969,"""AL""",4.092935,"""NO DH"""
2010,"""AL""",4.463719,"""DH"""
…,…,…,…
1973,"""AL""",4.301169,"""DH"""
2015,"""AL""",4.323618,"""DH"""
2001,"""NL""",4.753073,"""DH"""


In [14]:
from plotnine import ggplot, geom_line, aes

era_year = pitching.pivot(
    values = 'league_wide_ERA',
    index = 'yearID',
    on = 'lgID'
)
era_year = era_year.with_columns(
    (pl.col('AL') - pl.col('NL')).alias('Difference')
)
Era = pitching.join(era_year.select(['yearID', 'Difference']), on ='yearID', how='left')
                                          

In [15]:
from plotnine import ggplot, geom_line, geom_vline, annotate, theme_classic, theme, ggtitle, element_rect, labs, scale_color_manual, geom_hline, aes
Era = Era.with_columns(
    pl.lit('AL - NL').alias('Diff_Label')
)


Era_plot=(ggplot(data = Era)
 + geom_line(aes(x = "yearID",
                 y = "league_wide_ERA",
                 color = "lgID"
                )
            )
 + geom_line(aes(y = 'yearID',
                 x = 'Difference',
                 color = 'Diff_Label'
                ), size = 1)
 + geom_vline(xintercept = 1973, linetype = 'dashed', color = 'orange')
 + geom_vline(xintercept = 2022, linetype = 'dashed', color = 'orange')
 + annotate('text',
                x= 1950,
                y= 1.5,
                size = 8,
                label = 'DH was indroduced to the AL in 1973',
                ha = 'left')
 + annotate('text',
                x= 2022,
                y= 1.5,
                size = 8,
                label = 'DH was indroduced to the AL in 1973',
                ha = 'right')
 + theme_classic()
 + theme(
         panel_border = element_rect(color = 'black', fill = None, size = 1.5)
 )
 + ggtitle("League Era vs. YearID")
 + labs(
     x = "Year",
     y = "League ERA"
 )
 + geom_hline(yintercept = 0, linetype = 'dashed', color = 'grey')
 + scale_color_manual(name = "League",
                      values = {'AL' : 'blue',
                                'NL' : 'green',
                                'AL - NL' : 'red'}
                     )
 )



In [38]:
#from matplotlib import pyplot as plt
from plotnine import ggsave

print(type(Era_plot))

ggsave( filename = "image/Era_plot.png", plot = Era_plot, dpi = 300, width = 10, height = 6)

#Era_plot.save("image/Era_plot.png", dpi = 300, width = 10, height = 6)

<class 'plotnine.ggplot.ggplot'>


TypeError: ggplot.save() missing 1 required positional argument: 'self'

In [16]:
(batting :=
 pl.read_csv("./data/league_slg.csv")
)

yearID,lgID,league_wide_SLG,DH Rule
i64,str,f64,str
1979,"""AL""",0.408388,"""DH"""
1972,"""AL""",0.343013,"""NO DH"""
1958,"""NL""",0.405144,"""NO DH"""
2017,"""AL""",0.428764,"""DH"""
2012,"""AL""",0.410992,"""DH"""
…,…,…,…
2008,"""NL""",0.41289,"""DH"""
1987,"""AL""",0.425487,"""DH"""
1971,"""NL""",0.366387,"""NO DH"""


In [26]:
from plotnine import ggplot, geom_line, aes, geom_vline, annotate, theme_classic, theme, ggtitle, element_rect, scale_color_manual, geom_hline

SLG_year = batting.pivot(
    values = 'league_wide_SLG',
    index = 'yearID',
    on = 'lgID'
)
SLG_year = SLG_year.with_columns(
    (pl.col('AL') - pl.col('NL')).alias('SLG_Difference')
)
SLG_year = batting.join(SLG_year.select(['yearID', 'SLG_Difference']), on ='yearID', how='left')
                                          
SLG_plot = (ggplot(data = batting)
 + geom_line(aes(x = "yearID",
                 y = "league_wide_SLG",
                 color = "lgID"
                )
            )
 + geom_line(aes(y = 'yearID',
                 x = 'SLG_Difference',
                 color = 'DL_SLG'
                ), size = 1
            )
 + geom_vline(xintercept = 1973, linetype = 'dashed', color = 'orange')
 + geom_vline(xintercept = 2022, linetype = 'dashed', color = 'orange')
 + annotate('text', x = 1950,
                    y = 0.55,
                      size = 8,
                      label = 'DH was introduced to the NL in 2022',
                      ha = 'right')
 + theme_classic()
 + theme(
         panel_border = element_rect(color = 'black', fill = None, size = 1.5)
         )
 + ggtitle("League SLG vs. YearID")
 + geom_hline(yintercept = 0, linetype = 'dashed', color = 'grey')
 + scale_color_manual(name = "League",
                      values = {'AL' : 'blue',
                                'NL' : 'green',
                                'AL - NL' : 'red'}
                     )
)



In [27]:
print(SLG_plot)

<ggplot: (640 x 480)>


In [36]:
SLG_plot.save("image/SLG_plot.png", dpi=300, width = 10, height = 6)

C:\Users\rt2332in\AppData\Local\anaconda3\envs\polars\Lib\site-packages\plotnine\ggplot.py:630: PlotnineWarning: Saving 10 x 6 in image.
C:\Users\rt2332in\AppData\Local\anaconda3\envs\polars\Lib\site-packages\plotnine\ggplot.py:631: PlotnineWarning: Filename: image/SLG_plot.png


ModuleNotFoundError: No module named 'pyarrow'